In [1]:
import time
import pandas as pd
import numpy as np
from pandas import DataFrame
from datetime import datetime
import math

_time = time.time()
xls = pd.ExcelFile("..\Input file\Costing Examples with rates.xlsx")
def date_to_str(date):
    date = str(date).split("-")
    return date[1] + "/" + date[2] 

def day_val(day):
    val = {"Monday":0,"Tuesday":1,"Wednesday":2,'Thursday':3,"Friday":4,"Saturday":5,"Sunday":6}
    return val[day]

def get_excel():
    time_df = pd.read_excel(xls, 'Timebands').sort_values(["contactid","positionid"])
    return time_df
def get_rate_df():
    rate_df = pd.read_excel(xls, 'Rates').sort_values(["contactid","positionid","effectivedate"])
    return rate_df
def get_holiday():

    holiday_df = xls.parse("Holiday")
    holiday_list = [("" if m>9 else "0") + str(m) + "/" + ("" if d>9 else "0") + str(d) for d, m in zip(holiday_df["Day"], holiday_df["Month"])]

    return holiday_list

def get_holiday_rate():
    holiday_rate = 2.5
    return holiday_rate

def get_rate_data():
    rate1 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
    rate2 = np.array([[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1,1.15,1.25,1.5,2],[1.5,1.5,1.5,1.5,2],[2,2,2,2,2]])
    rate3 = np.array([[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.2,1.35,1.45,1.7,2.2],[1.7,1.7,1.7,1.7,2.2],[2.2,2.2,2.2,2.2,2.2]])   
    rate_data = {"fulltime": rate1, "parttime": rate2, "casual": rate3}
    return rate_data



In [2]:
def get_base_rate():
    rate_df = get_rate_df()
    time_df = get_excel()

    check_null = rate_df["contactid"].isnull() & rate_df["positionid"].isnull()

    amount_list=[]
    
    def foo(row):        
        _contact= row['contactid']
        _position = row['positionid'] 
        _start = row['RoundedStart']        
        rate = 0        
        check_effdate = rate_df["effectivedate"] <= _start
        get_amount = []
        rate = 0
        if isinstance(_position, str):
            check_position = rate_df["positionid"] == _position
            if len(set(check_position))==1: # position not found 
                if isinstance(_contact, str):
                    check_contact = rate_df["contactid"] == _contact
                    if len(set(check_contact))==2: # contact found
                        get_amount = list(rate_df["amount"][check_contact & check_effdate])
                        rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                    else:
                        get_amount = list(rate_df["amount"][check_null & check_effdate])
                        rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                else: # contact is null
                    get_amount = list(rate_df["amount"][check_null & check_effdate])                    
                    rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                        
            else: # position found
                check_contact = rate_df["contactid"] == _contact
                if isinstance(_contact, str): # contact not null
                    if len(set(check_contact))==2: # contact found
                        get_amount = list(rate_df["amount"][check_position & check_contact & check_effdate])
                        if len(get_amount) == 0: # both match but not in the same row
                            get_amount = list(rate_df["amount"][check_position & check_effdate])
                            rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                        else:
                            rate = float(get_amount[-1])
                            
                    else: # contact not found
                        get_amount = list(rate_df["amount"][check_position & check_effdate])
                        rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                else: # contact null
                    get_amount = list(rate_df["amount"][check_position & check_effdate])
                    rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
        else: # position is null
            if isinstance(_contact, str): # contact found
                check_contact = rate_df["contactid"] == _contact
                if len(set(check_contact))==2: # contact found
                    get_amount = list(rate_df["amount"][check_contact & check_effdate])
                    rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                else:
                    get_amount = list(rate_df["amount"][check_null & check_effdate])
                    rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
            else: # contact is null
                get_amount = list(rate_df["amount"][check_null & check_effdate])
                rate = float(get_amount[-1]) if len(get_amount) > 0 else 0
                   
        return rate 
    amount_list = time_df.apply(foo, axis=1)  
    return amount_list



In [4]:
holiday_list = get_holiday()
holiday_rate = get_holiday_rate()
rate_data = get_rate_data()
def cal_base_rate(_type, day, start_time, end_time, rate_data,start_date,end_date):
    shift = 2      
    ord_rate = 0
    ot1_rate = 0
    ot2_rate = 0
    if date_to_str(start_date) in holiday_list:
        ord_rate = holiday_rate
        ot1_rate = holiday_rate
        ot2_rate = holiday_rate
    else:        
        if _type == "fulltime":
            if start_time < 5 or (end_time > 1 and start_date != end_date):
                shift = 2
            elif end_time > 19 or (end_time < 1 and start_date != end_date):
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        elif _type == "parttime":
            
            if start_time < 5 or (end_time > 1 and start_date != end_date):
                shift = 2
            elif end_time > 19 or (end_time < 1 and start_date != end_date):
                shift = 1
                             
            elif start_time >= 5 and end_time < 20:
                shift = 0

        else:
            
            if start_time < 5 or (end_time > 1 and start_date != end_date):
                shift = 2
            elif end_time > 19 or (end_time < 1 and start_date != end_date):
                shift = 1
            elif start_time >= 5 and end_time < 20:
                shift = 0
            
        ord_rate = rate_data[_type][day_val(day)][shift]
        ot1_rate = rate_data[_type][day_val(day)][3]
        ot2_rate = rate_data[_type][day_val(day)][4]
    return (ord_rate,ot1_rate,ot2_rate,shift)

def get_maxot1(_type):
    if _type == "casual":
        maxot_1 = 10
    elif _type == "fulltime" or _type == "parttime":
        maxot_1 = 8
    return maxot_1

time_df = get_excel()

def get_data(time_df):
    time_df['Amount'] = get_base_rate()
    import re
    data= []
    last_data = []
    for row in time_df.iterrows():    
        _rate = row[1]['Amount']
        _id = row[1]['contactid']
        positionid= row[1]['positionid']
        objecttimeid = row[1]['objecttimeid']
        parentotid = row[1]['ParentOTID']
        if type(parentotid) == float:
            parentotid = objecttimeid
        
        start_period = row[1]['PeriodStartDate'].date()   
        end_period = row[1]['PeriodEndDate'].date()
        start = row[1]["RoundedStart"]
        end   = row[1]["RoundedEnd"]   
        start_time = start.hour + start.minute/60
        end_time = end.hour + end.minute/60
    
        _type = row[1]["employmenttype"]
        for k in _type.lower().split('\n'):
            _type=re.sub(r"[^a-zA-Z0-9]+",'',k).replace("temporary","")
            
        start_date = start.date() 
        end_date   = end.date()
        
        start_day  = start.strftime('%A')
        end_day    = end.strftime('%A')  
            
        if ((start_date != end_date) and ( ((end_time >1) and (start_day in ["Friday","Saturday","Sunday"]))  or ( date_to_str(start_date) in holiday_list or date_to_str(end_date) in holiday_list ) )):     
            delta_time1 = 24 - start_time 
            delta_time2 = end_time      
            start_date1 = start_date   
            end_date1= end_date
            start_date2 = end_date
            end_date2= end_date
        
            start1= start
            end2=end
            end1= end.replace(hour=0,minute=0)
            start2=end1
            start_day1=start1.strftime('%A')
         
            start_day2=start2.strftime('%A')
            
            data.append((parentotid,start_day1, start1, end1, _type))
            data.append((parentotid,start_day2, start2, end2,_type))  
            last_data.append((_id,positionid,start_period,end_period,parentotid,objecttimeid, start1, end1,start_day1, _type,delta_time1,_rate))
            last_data.append((_id,positionid,start_period,end_period,parentotid, objecttimeid,start2, end2,start_day2, _type,delta_time2,_rate))
            
        else: 
            delta_time = (end-start).total_seconds() / 3600.0
         
            data.append((parentotid,start_day,start,end,_type))
            last_data.append((_id,positionid,start_period,end_period,parentotid, objecttimeid,start, end,start_day, _type,delta_time,_rate))
        
    file = pd.DataFrame(data,columns=["ParentOT ID",'Day','Start Date','End Date','Type'])
    for parrent_id in file["ParentOT ID"].unique():
        
        last_end_date = file[file["ParentOT ID"] == parrent_id]['End Date'].iloc[-1]
        last_start_date = file[file["ParentOT ID"] == parrent_id]['Start Date'].iloc[0]

        if  len(file[file["ParentOT ID"] == parrent_id]["Day"].unique())>1      :
            file.loc[file["ParentOT ID"] == parrent_id,"End Date"] = file['End Date']
            file.loc[file["ParentOT ID"] == parrent_id,"Start Date"] = file['Start Date']
        else:
            file.loc[file["ParentOT ID"] == parrent_id,"End Date"] = last_end_date 
            file.loc[file["ParentOT ID"] == parrent_id,"Start Date"] = last_start_date
            
    file['Start Time'] = [i.hour + i.minute/60 for i in file['Start Date']]
    file['End Time'] = [i.hour + i.minute/60 for i in file['End Date']] 
    last_file = pd.DataFrame(last_data,columns=["ContactID",'PositionID','Start Period','End Period','Parent ID','Object ID','Shift Start','Shift End','Day','Type','Quantity','Amount'])
    last_file['Factor'] = [cal_base_rate(file['Type'][i],file['Day'][i],file['Start Time'][i],file['End Time'][i], rate_data,file['Start Date'][i].date(),file['End Date'][i].date())[0] for i in range(len(file['Day']))]
    last_file['Rate OT1'] = [cal_base_rate(file['Type'][i],file['Day'][i],file['Start Time'][i],file['End Time'][i], rate_data,file['Start Date'][i].date(),file['End Date'][i].date())[1] for i in range(len(file['Day']))]
    last_file['Rate OT2'] = [cal_base_rate(file['Type'][i],file['Day'][i],file['Start Time'][i],file['End Time'][i], rate_data,file['Start Date'][i].date(),file['End Date'][i].date())[2] for i in range(len(file['Day']))]
    last_file['Shift'] = [cal_base_rate(file['Type'][i],file['Day'][i],file['Start Time'][i],file['End Time'][i], rate_data,file['Start Date'][i].date(),file['End Date'][i].date())[3] for i in range(len(file['Day']))]
    last_file['shift'] = ['Day shift' if last_file['Shift'][i] == 0 else ('Afternoon shift' if last_file['Shift'][i] ==1 else 'Night shift' ) for i in range(len(last_file['Day']))]
    last_file['TransactionTypeName'] = ['Holiday shift' if last_file['Factor'][i]==2.5 else   ( last_file['Day'][i] if last_file['Day'][i] in ['Saturday','Sunday'] else (last_file['shift'][i])) for i in range(len(last_file['Day']))]
    last_file['Start Date'] = file['Start Date']
    last_file['End Date'] = file['End Date']
    
#     table = pd.DataFrame(last_file).drop(['Shift', 'shift','Day'], axis='columns')#.sort_values(["ContactID",'Start Period','Start Date'])
    return last_file.drop(['Shift', 'shift','Day'], axis='columns')      

get_data(time_df)

,ContactID,PositionID,Start Period,End Period,Parent ID,Object ID,Shift Start,Shift End,Type,Quantity,Amount,Factor,Rate OT1,Rate OT2,TransactionTypeName,Start Date,End Date
0,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,35487CEB-9FB1-4566-A795-D93A932C66D2,35487CEB-9FB1-4566-A795-D93A932C66D2,2021-04-05 08:00:00,2021-04-05 08:05:00,casual,0.083333,32.6,1.2,1.7,2.2,Day shift,2021-04-05 08:00:00,2021-04-05 08:05:00
1,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,3A66FAD0-70E4-4214-A081-D44DEEC14A0A,3A66FAD0-70E4-4214-A081-D44DEEC14A0A,2021-04-06 08:00:00,2021-04-06 08:05:00,casual,0.083333,32.6,1.2,1.7,2.2,Day shift,2021-04-06 08:00:00,2021-04-06 08:05:00
2,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,C5DA97FB-1DB4-4047-A557-53BA6B006249,C5DA97FB-1DB4-4047-A557-53BA6B006249,2021-04-07 06:00:00,2021-04-07 14:30:00,casual,8.500000,32.6,1.2,1.7,2.2,Day shift,2021-04-07 06:00:00,2021-04-07 14:30:00
3,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,3EBA273A-5A2C-4BE1-8D08-15F9A56972EF,3EBA273A-5A2C-4BE1-8D08-15F9A56972EF,2021-04-08 06:00:00,2021-04-08 14:00:00,casual,8.000000,32.6,1.2,1.7,2.2,Day shift,2021-04-08 06:00:00,2021-04-08 14:00:00
4,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2FFAAB5D-4303-4D79-A75E-BF70F341DE43,2FFAAB5D-4303-4D79-A75E-BF70F341DE43,2021-04-09 08:00:00,2021-04-09 08:05:00,casual,0.083333,32.6,1.2,1.7,2.2,Day shift,2021-04-09 08:00:00,2021-04-09 08:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2391,FF3D0B2A-0961-47E3-9787-568766EDB14F,89E9CEDF-A028-48C0-A1D5-F22888FDAE81,2021-04-26,2021-05-02,114E6171-33B4-4079-90BC-9AD3FBCE98EC,D3ACAD4D-B850-4ABA-8406-0ADE0841BCE1,2021-04-30 13:30:00,2021-04-30 17:00:00,casual,3.500000,36.2,2.5,2.5,2.5,Holiday shift,2021-04-30 09:00:00,2021-04-30 17:00:00
2392,FF3D0B2A-0961-47E3-9787-568766EDB14F,89E9CEDF-A028-48C0-A1D5-F22888FDAE81,2021-04-26,2021-05-02,26BDD246-2914-4D43-92B3-5056C6171171,26BDD246-2914-4D43-92B3-5056C6171171,2021-05-01 09:00:00,2021-05-01 13:00:00,casual,4.000000,36.2,1.7,1.7,2.2,Saturday,2021-05-01 09:00:00,2021-05-01 17:15:00
2393,FF3D0B2A-0961-47E3-9787-568766EDB14F,89E9CEDF-A028-48C0-A1D5-F22888FDAE81,2021-04-26,2021-05-02,26BDD246-2914-4D43-92B3-5056C6171171,338AFC37-C601-4C31-A229-189E5403834F,2021-05-01 13:30:00,2021-05-01 17:15:00,casual,3.750000,36.2,1.7,1.7,2.2,Saturday,2021-05-01 09:00:00,2021-05-01 17:15:00
2394,FF3D0B2A-0961-47E3-9787-568766EDB14F,89E9CEDF-A028-48C0-A1D5-F22888FDAE81,2021-05-03,2021-05-09,8BEADDE4-74B7-422A-9121-305EF8B0829D,8BEADDE4-74B7-422A-9121-305EF8B0829D,2021-05-07 09:00:00,2021-05-07 13:00:00,casual,4.000000,36.2,1.2,1.7,2.2,Day shift,2021-05-07 09:00:00,2021-05-07 15:00:00


In [5]:

def cal_payment():
    file = get_data(time_df)
    period_hour = {}
    parent_hour = {}
    
    shift_pay = []
    hour_day_list = {}
    
    for row in file.iterrows():
        _parent = row[1]["Parent ID"]
        _id          = row[1]["ContactID"]
        _key_parent = str(_id) + " " + str(_parent)
        _hour = row[1]["Quantity"]  
        if _key_parent in hour_day_list:
            hour_day_list[_key_parent] += _hour
        else:
            hour_day_list[_key_parent]= _hour
        
    for row in file.iterrows():
        _id          = row[1]["ContactID"]
        _period      = row[1]["Start Period"] 
        _key_period  = str(_id) + " " + str(_period)
        _hour = row[1]["Quantity"]   
        _rate = row[1]["Factor"]
        _ot1_rate = row[1]["Rate OT1"]
        _ot2_rate = row[1]["Rate OT2"]
        _type = row[1]["Type"]
        _parent = row[1]["Parent ID"]
        _object_id = row[1]["Object ID"]
        _key_parent = str(_id) + " " + str(_parent)
        base_rate = row[1]['Amount']
        payment = 0
        maxot_1 = get_maxot1(_type) 
        

        total_hour_parent = parent_hour[_key_parent] if _key_parent in parent_hour else 0      

        total_hour_period = period_hour[_key_period] if _key_period in period_hour else 0

        if (total_hour_parent + _hour <= maxot_1): 
            _hour_new = _hour + ((3 - hour_day_list[_key_parent]) if list(file[file["Parent ID"] ==_parent]['Object ID'])[-1] == _object_id and hour_day_list[_key_parent] < 3 else 0)

            if total_hour_period + _hour <= 38:
                payment = base_rate * _hour_new * _rate
                
            elif total_hour_period >= 40 :
              
                payment = base_rate * _hour_new * _ot2_rate
                
            elif total_hour_period >=38 and total_hour_period +  _hour < 40:
               
                payment = base_rate * _hour_new * _ot1_rate

            elif total_hour_period >=38 and total_hour_period +  _hour >= 40:  
                payment = base_rate*((40-total_hour_period)*_ot1_rate + (_hour_new + total_hour_period - 40)*_ot2_rate)

            elif total_hour_period + _hour_new  >  38 and total_hour_period < 38:
                ot_hour = (total_hour_period + _hour) - 38 

                base_hour = 38 - total_hour_period 
            

                base_pay = base_rate*(base_hour * _rate)

                ot_pay = base_rate*(min(2,ot_hour)*_ot1_rate + max(0,ot_hour-2)*_ot2_rate)

                payment = base_pay + ot_pay

            total_hour_period += _hour
            total_hour_parent += _hour
            
        else:   
            
            if total_hour_period + min(_hour,maxot_1) <= 38:   

                _hour_new = maxot_1 - total_hour_parent
                _hour_ot = total_hour_parent + _hour - maxot_1
                payment = (_hour_new * _rate + min(2, _hour_ot) * _ot1_rate + max(_hour_ot - 2, 0) * _ot2_rate)*base_rate
                
            elif total_hour_period >= 40 :              
                payment = base_rate*(max(_hour,3)*_ot2_rate)

            elif total_hour_period >=38 and total_hour_period + _hour <40:               
                payment = base_rate*(3*_ot1_rate)

            elif total_hour_period >=38 and total_hour_period + _hour >=40:    
                payment = base_rate*((40-total_hour_period)*_ot1_rate + (max(_hour,3) + total_hour_period-40)*_ot2_rate)

            elif total_hour_period + min(_hour,maxot_1)  >  38 and total_hour_period < 38:
                ot_hour = (total_hour_period + _hour) - 38 

                base_hour = 38 - total_hour_period 
            
                base_pay = base_rate*(base_hour *_rate)

                ot_pay = base_rate*(min(2,ot_hour)*_ot1_rate + max(0,ot_hour-2)*_ot2_rate)

                payment = base_pay + ot_pay
            
            total_hour_period += (maxot_1 - total_hour_parent) if total_hour_parent < maxot_1 else 0
            total_hour_parent += min(_hour,maxot_1) 
        
        parent_hour[_key_parent] = total_hour_parent
      
        period_hour[_key_period] = total_hour_period 
    
        shift_pay.append(payment)
    
    file["Payment"] = shift_pay
    
    return file.drop(['Rate OT1', 'Rate OT2',"Parent ID","Object ID"], axis='columns')
output = cal_payment()

result = output.reset_index(drop=True)
result.head(30)
# result = result.to_excel("report_31_05.xlsx", index = True)
# print("Time elapsed: ", time.time() -  _time) 

,ContactID,PositionID,Start Period,End Period,Shift Start,Shift End,Type,Quantity,Amount,Factor,TransactionTypeName,Start Date,End Date,Payment
0,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-05 08:00:00,2021-04-05 08:05:00,casual,0.083333,32.60,1.20,Day shift,2021-04-05 08:00:00,2021-04-05 08:05:00,117.360000
1,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-06 08:00:00,2021-04-06 08:05:00,casual,0.083333,32.60,1.20,Day shift,2021-04-06 08:00:00,2021-04-06 08:05:00,117.360000
2,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-07 06:00:00,2021-04-07 14:30:00,casual,8.500000,32.60,1.20,Day shift,2021-04-07 06:00:00,2021-04-07 14:30:00,332.520000
3,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-08 06:00:00,2021-04-08 14:00:00,casual,8.000000,32.60,1.20,Day shift,2021-04-08 06:00:00,2021-04-08 14:00:00,312.960000
4,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-09 08:00:00,2021-04-09 08:05:00,casual,0.083333,32.60,1.20,Day shift,2021-04-09 08:00:00,2021-04-09 08:05:00,117.360000
5,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-10 08:00:00,2021-04-10 08:05:00,casual,0.083333,32.60,1.70,Saturday,2021-04-10 08:00:00,2021-04-10 08:05:00,166.260000
6,00607537-26E5-4A38-AEF0-2D80F50D8000,166F77B4-A356-40C6-B792-969359A16034,2021-04-05,2021-04-11,2021-04-11 08:00:00,2021-04-11 08:05:00,casual,0.083333,32.60,2.20,Sunday,2021-04-11 08:00:00,2021-04-11 08:05:00,215.160000
7,05E2A0B2-B0EA-43A6-99C6-886FCC1DE38F,11EA9868-A6AC-4D68-B842-1471C5656348,2021-05-10,2021-05-16,2021-05-10 07:00:00,2021-05-10 17:30:00,casual,10.500000,23.83,1.20,Day shift,2021-05-10 07:00:00,2021-05-10 17:30:00,306.215500
8,05E2A0B2-B0EA-43A6-99C6-886FCC1DE38F,11EA9868-A6AC-4D68-B842-1471C5656348,2021-05-10,2021-05-16,2021-05-11 07:00:00,2021-05-11 17:30:00,casual,10.500000,23.83,1.20,Day shift,2021-05-11 07:00:00,2021-05-11 17:30:00,306.215500
9,05E2A0B2-B0EA-43A6-99C6-886FCC1DE38F,11EA9868-A6AC-4D68-B842-1471C5656348,2021-05-10,2021-05-16,2021-05-12 07:00:00,2021-05-12 17:30:00,casual,10.500000,23.83,1.20,Day shift,2021-05-12 07:00:00,2021-05-12 17:30:00,306.215500
